## Generating 'average waiting time in units service time' tables

This notebook uses the multi thread engine of the OpenQTSim package to start up multiple processes for the parallel processing of a list of tasks. 

This particular example notebook produces the typical 'average waiting time in units service time' tables that are presented in the PIANC 2014 guideline for example. You can experiment with different distributions for the interarrival and service times. Currently implemented are:
* 'M': Exponential distribution, 
* 'E2': Erlang 2 distribution.

NB: the table becomes more accurate when a larger number of arrivals (nr_arr) is simulated. Obviously there is a tradeoff between accuracy and calculation time.

### Create task list and perform calculations

In [5]:
%%time
if __name__ == '__main__': # protect your program's entry point
    
    import pandas as pd
    import numpy as np
    import openqtsim

    from multiprocessing import Pool
    from openqtsim.mt_engine import worker, Task

    # every instance will be run with nr_arr number of customers
    nr_arr = 2000     # nr of customers to simulate
    A = 'M'           # distribution of inter arrival times (M: exponential, E2: erlang 2)
    S = 'M'           # distribution of service times (M: exponential, E2: erlang 2)
    nr_servers = 2    # nr of servers to consider (1 to nr_servers)

    # --- servers to loop through ---
    # make an array of the nr of servers to loop through
    servers =  np.array(range(1, nr_servers + 1))

    # --- utilisations to loop through ---
    # lambdas are selected to be ranging from 1 to 9
    lambdas = np.array(range(1,10))

    # mu is selected to always be 10 
    mu=10
    
    # the lambdas divided by mu provide the utilisations to loop through
    utilisations = lambdas / mu

    tasks=[]
    # one by one add tasks to the 'tasks' list
    for index, lam in enumerate(lambdas):
        for c in servers:
            tasks.append(Task(A, S, c, nr_arr, lam, mu))

    # select nr of processes that will carry out tasks submitted to Pool
    p = Pool(4)
    
    # map the worker with tasks list to the Pool 
    result = p.map(worker, tasks)
    
    # close processes in the Pool
    p.close()

Wall time: 10.2 s


### Reshape 'result' variable to the desired table format

In [6]:
# organise results into a table of waiting times as a factor of service time
df = pd.DataFrame(tasks)
df['result'] = result
df['utilisation'] = df.lam / df.mu

df.pivot('utilisation', 'c', values='result')

c,1,2
utilisation,,
0.1,0.101460,0.010111
0.2,0.236858,0.044828
0.3,0.443159,0.115721
0.4,0.706875,0.210007
0.5,0.979540,0.280166
0.6,1.636729,0.614975
0.7,2.242476,0.957473
0.8,3.809454,1.714403
0.9,7.229202,2.840920
